In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_16_2/checkpoint-50/"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, device_map="auto")

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    with torch.no_grad():
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        
        val = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return val, input_ids

In [3]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

Generating train split:   0%|          | 0/910 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/195 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/195 [00:00<?, ? examples/s]

In [5]:
val = 0
import json
for i in range(3,53):
    data = json.loads(dataset['test'][i]['text'])
    val += (len(data['dialog']))

print(val)

1590


In [24]:
print(json.loads(dataset['test'][0]['text'])['dialog'])

[{'text': 'Hello. How are you today?', 'speaker': 'sys', 'strategy': 'Others'}, {'text': 'hi i am okay, a little bit sad though', 'speaker': 'usr'}, {'text': 'Okay. I am very sorry to hear that! Do you want to tell me more about that?', 'speaker': 'sys', 'strategy': 'Question'}, {'text': 'Well with the holidays coming up i have been very stressed and nervous about what i am going to do', 'speaker': 'usr'}, {'text': 'I really feel you there. Holidays are so hard.. especially with the way this year has been. Anything specific?', 'speaker': 'sys', 'strategy': 'Affirmation and Reassurance'}, {'text': "Yes. I am a college student and I live on campus. The day before I moved in my dad said some unforgivable things to me and we haven't spoke since", 'speaker': 'usr'}, {'text': 'I have no choice but to go live at home over the break but I am very scared about it', 'speaker': 'usr'}, {'text': "Do you feel scared for your own self? Or of the situation in general? I'm sorry to hear that. Relation

In [9]:
import spacy

nlp = spacy.load('en_core_web_sm')

included_tags = {"NOUN", "VERB", "INTJ" "ADJ", "ADV", "ADP", "PROPN", "NUM", "SCONJ", "PRON"}
def get_sentence(sentence):
    new_sentence = []
    for token in nlp(sentence):
        #print(token.pos_)
        if token.pos_ in included_tags:
            new_sentence.append(token.text)
    res = " ".join(new_sentence)
    return res

#get_sentence(data[0])


In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    #print("stopwords")
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from tqdm import tqdm

max_length = 1024
stride = 128

#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024

In [12]:
messages_final = []
messages_no_sw_final = []
messages_boh_final = []
messages_no_sw_original_final = []
messages_boh_original_final = []

perplexity_final, perplexity_no_sw_final, perplexity_boh_final = [], [], []
nlls_final, nlls_no_sw_final, nlls_boh_final = [], [], []
prev_end_loc = 0
for num in range(3, 50):
    #print(data)
    messages = []
    messages_no_sw = []
    messages_no_sw_original = []
    messages_boh = []
    messages_boh_original = []
    
    nlls, nlls_boh, nlls_no_sw = [], [], []
    
    perplexity, perplexity_no_sw, perplexity_boh = [], [], []
    
    data = json.loads(dataset['test'][num]['text'])
    data = data['dialog']
    print(data)
    try:
        for i in range(0, len(data), 1):
            for begin_loc in (range(0, seq_len, stride)):
                #print(type(data[i]['content']))
                
                if data[i]['speaker'] == 'sys':
                    break
                
                messages.append({"role": "user", "content":  data[i]['text']})
                messages_no_sw.append({"role": "user", "content": remove_stopwords(data[i]['text'])})
                messages_no_sw_original.append({"role": "user", "content": data[i]['text']})
                messages_boh.append({"role": "user", "content": get_sentence(data[i]['text'])})
                messages_boh_original.append({"role": "user", "content": data[i]['text']})
                
                #i+=2
                
                end_loc = min(begin_loc + max_length, seq_len)
                trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
                #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
                input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_no_sw = tokenizer.apply_chat_template(messages_no_sw, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=False, add_generation_prompt=True, return_tensors="pt").to("cuda")
                target_ids = input_ids.clone()
                target_ids_boh = input_ids_boh.clone()
                target_ids_no_sw = input_ids_no_sw.clone()
                target_ids[:, :-trg_len] = -100
                target_ids_boh[:, :-trg_len] = -100
                target_ids_no_sw[:, :-trg_len] = -100
                
                with torch.no_grad():
                    val, _ = generate_content(tokenizer, messages)
                    val_no_sw, _ = generate_content(tokenizer, messages_no_sw)
                    val_boh, _ = generate_content(tokenizer, messages_boh)
                    #print(val)
                    outputs = model(input_ids=input_ids, labels=target_ids)
                    outputs_boh = model(input_ids=input_ids_boh, labels=target_ids_boh)
                    outputs_no_sw = model(input_ids=input_ids_no_sw, labels=target_ids_no_sw)
                    
                    messages.append({"role": "assistant", "content": val})
                    messages_no_sw.append({"role": "assistant", "content": remove_stopwords(val_no_sw)})
                    messages_no_sw_original.append({"role": "assistant", "content": val_no_sw})
                    messages_boh.append({"role": "assistant", "content": get_sentence(val_boh)})
                    messages_boh_original.append({"role": "assistant", "content": val_boh})
                    
                    neg_log_likelihood = outputs.loss
                    neg_log_likelihood_boh = outputs_boh.loss
                    neg_log_likelihood_no_sw = outputs_no_sw.loss
                    
                nlls.append(neg_log_likelihood)
                nlls_boh.append(neg_log_likelihood_boh)
                nlls_no_sw.append(neg_log_likelihood_no_sw)
                
                prev_end_loc = end_loc
                if end_loc == seq_len:
                    break
                    
            ppl = torch.exp(torch.stack(nlls).mean())
            ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
            ppl_no_sw = torch.exp(torch.stack(nlls_no_sw).mean())
            print(ppl)
            print(ppl_boh)
            print(ppl_no_sw)
            perplexity.append(ppl)
            perplexity_boh.append(ppl_boh)
            perplexity_no_sw.append(ppl_no_sw)
            
    except Exception as e:
        print(e)
        pass
        
    messages_final.append(messages)
    messages_no_sw_final.append(messages_no_sw)
    messages_no_sw_original_final.append(messages_no_sw_original)
    messages_boh_final.append(messages_boh)
    messages_boh_original_final.append(messages_boh_original)
    nlls_final.append(nlls)
    nlls_no_sw_final.append(nlls_no_sw)
    nlls_boh_final.append(nlls_boh)
    perplexity_final.append(perplexity)
    perplexity_no_sw_final.append(perplexity_no_sw)
    perplexity_boh_final.append(perplexity_boh)
                
            
            

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'text': 'Hello', 'speaker': 'usr'}, {'text': 'Good Afternoon', 'speaker': 'sys', 'strategy': 'Others'}, {'text': 'How are you doing troday?', 'speaker': 'sys', 'strategy': 'Question'}, {'text': "I am Ok thanks but have an unusual issue. I think my girlfriend may be cheating on me but I'm too scared to do anything about it as I don't want her to leave me. I feel ashamed of myself but I can't help it", 'speaker': 'usr'}, {'text': 'Being in a relationship with someone when you suspect unfaithful behavior is a rough thing to go through.', 'speaker': 'sys', 'strategy': 'Affirmation and Reassurance'}, {'text': "Yes it is but I should be leaving her and moving on. Problem is she is a 9 and I'm a 5 and I love her. Pretty sure she is cheating though so hopefully if I ignore it she may just leave things as they are.", 'speaker': 'usr'}, {'text': "I've dealt with the same problem with my partner more than once. I love him very much and I found him to be a 9 to my 5 as well. However, the questio

In [14]:
with open ('data/messages_final.json', 'w') as f:
    json.dump(messages_final, f)
    
with open ('data/messages_no_sw_final.json', 'w') as f:
    json.dump(messages_no_sw_final, f)

with open ('data/messages_no_sw_original_final.json', 'w') as f:
    json.dump(messages_no_sw_original_final, f)

with open ('data/messages_boh_final.json', 'w') as f:
    json.dump(messages_boh_final, f)
    
with open ('data/messages_boh_original_final.json', 'w') as f:
    json.dump(messages_boh_original_final, f)
    
with open('data/test_data.json', 'w') as f:
    json.dump(data, f)
    
torch.save(nlls_final, 'data/nlls_final.pt')
torch.save(nlls_no_sw_final, 'data/nlls_no_sw_final.pt')
torch.save(nlls_boh_final, 'data/nlls_boh_final.pt')
torch.save(perplexity_final, 'data/perplexity_final.pt')
torch.save(perplexity_no_sw_final, 'data/perplexity_no_sw_final.pt')
torch.save(perplexity_boh_final, 'data/perplexity_boh_final.pt')